In [2]:
import cv2
import mediapipe as mp
import csv
import time
import numpy as np
from datetime import datetime

In [41]:
# === CONFIGURACIÓN DEL USUARIO ===
LABEL = "Pueblo"              # 👈 Cambia la etiqueta según la seña
OUTPUT_FILE = "dataset.csv"
NUM_SAMPLES = 200        # 👈 Cuántos ejemplos por seña quieres

# Configuración de MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# === Función: convertir landmarks a vector normalizado (63 floats por mano) ===
def hand_to_feature_vector(hand_landmarks):
    coords = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark], dtype=np.float32)
    wrist = coords[0].copy()
    coords -= wrist
    scale = np.linalg.norm(coords[9])  # muñeca → base dedo medio
    if scale <= 1e-6:
        scale = np.max(np.linalg.norm(coords, axis=1))
        if scale <= 1e-6:
            scale = 1.0
    coords /= scale
    return coords.flatten().tolist()

# === Orden consistente de manos (izquierda → derecha) ===
def sort_hands_by_x(multi_hand_landmarks):
    hands = []
    for hand in multi_hand_landmarks:
        coords = np.array([[lm.x, lm.y] for lm in hand.landmark])
        centroid_x = coords[:, 0].mean()
        hands.append((centroid_x, hand))
    hands_sorted = [h for _, h in sorted(hands, key=lambda x: x[0])]
    return hands_sorted

# === Inicializar CSV ===
with open(OUTPUT_FILE, "a", newline="") as f:
    writer = csv.writer(f)
    if f.tell() == 0:  # Si el archivo está vacío, escribir cabecera
        header = [f"f{i}" for i in range(126)] + ["num_hands", "label", "timestamp"]
        writer.writerow(header)

# === Captura de cámara ===
cap = cv2.VideoCapture(0)

try:
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    ) as hands:

        sample_count = 0
        print(f"Grabando señas para: {LABEL}")
        time.sleep(2)

        while cap.isOpened() and sample_count < NUM_SAMPLES:
            success, frame = cap.read()
            if not success:
                continue

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            row = []
            num_hands = 0
            if results.multi_hand_landmarks:
                sorted_hands = sort_hands_by_x(results.multi_hand_landmarks)
                num_hands = len(sorted_hands)
                for hand_landmarks in sorted_hands:
                    row.extend(hand_to_feature_vector(hand_landmarks))
                    mp_drawing.draw_landmarks(frame_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Rellenar con ceros hasta 126 features
            while len(row) < 126:
                row.append(0.0)

            if any(val != 0.0 for val in row):  # si detectó al menos una mano
                row.append(num_hands)
                row.append(LABEL)
                row.append(datetime.now().isoformat())

                with open(OUTPUT_FILE, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(row)

                sample_count += 1

            cv2.putText(frame_bgr, f"Label: {LABEL}  Sample: {sample_count}/{NUM_SAMPLES}",
                        (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Recolector de Dataset", frame_bgr)

            if cv2.waitKey(1) & 0xFF == 27:  # ESC para salir
                break

        print(f"✅ Captura terminada para la seña {LABEL} ({sample_count} muestras)")
finally:
    cap.release()
    cv2.destroyAllWindows()


Grabando señas para: Pueblo
✅ Captura terminada para la seña Pueblo (200 muestras)
